# Build Evaluation Data

## 1. Setup
Ensure that the required environment variables are set.

### 1a. Load environment variables

In [ ]:
! pip install python-dotenv --quiet
from dotenv import load_dotenv
load_dotenv('../.env')

### 1b. Install pre-requisite libraries

In [ ]:
! pip install requests beautifulsoup4 markdownify urllib3 --quiet
! pip install ipykernel --quiet
! pip install ipywidgets --quiet
! pip install llama-index llama-index-vector-stores-azureaisearch --quiet
! pip install azure-search-documents --quiet
! pip install azure-data-tables --quiet
! pip install spacy --quiet

### 1c. Enable Async

In [ ]:
# NESTED ASYNCIO LOOP NEEDED TO RUN ASYNC IN A NOTEBOOK
import nest_asyncio
nest_asyncio.apply()

### 1d. Create JSONL workspace

In [ ]:
import os
folder_path = 'evaluation_output'
os.makedirs(folder_path, exist_ok=True)
llama_path = f'{folder_path}/llama_eval'
os.makedirs(llama_path, exist_ok=True)
pf_path = f'{folder_path}/pf_eval'
os.makedirs(pf_path, exist_ok=True)
pf_merged_path = f'{folder_path}/pf_merged_eval'
os.makedirs(pf_merged_path, exist_ok=True)

## 2. Load, Fetch and Evaluate Inventory Items

### 2a. Load Inventory Subset

In [ ]:
%run -i ../utilities/storeInventory.py
%run -i ../utilities/fetchAndIngest.py
%run -i ../utilities/jsonlConverter.py

# Print the number of rows in the inventory table
print(f'The inventory table contains {get_inventory_table_count()} rows.')

documents=[]

# Fetch rows 100 and 110 of the table & ingest the url
for inventory in get_inventory_entities_by_page(page_size=50, page_number=1):
    print(f'Fetching {inventory['Url']} ...')
    document = generate_llama_document(fetch_document(inventory['Url']))
    documents.append(document)

### 2b. (Optional) Evaluate the first inventory item

This creates an evaluation file in the JSONL format that will be consumed by Llama Index & Prompt Flow's evaluation flows.

In [ ]:
from llama_index.core.llama_dataset.generator import RagDatasetGenerator
from IPython.display import Markdown, JSON 

# document = documents[0]
# print(f'Evaluating {inventory['Url']} with id {document.doc_id}...')
# rag_data_generator = RagDatasetGenerator.from_documents(
#     documents=[document], 
#     show_progress=True,
#     llm=get_aoai()['llm'],
#     num_questions_per_chunk=1
# )
# dataset = rag_data_generator.generate_dataset_from_nodes()
# input_file = f"{llama_path}/{document.doc_id}.jsonl"
# output_file = f"{pf_path}/{document.doc_id}.jsonl"
# dataset.save_json(input_file)
# convert_llama_to_pf_jsonl(input_file, output_file)
# print(f'JSONL evaluation for LlamaIndex created at {input_file}')
# print(f'JSONL evaluation for PromptFlow created at {output_file}')
# display(JSON(f"{dataset.json()}"))

### 2c. (Optional) Evaluate the inventory list

This creates an evaluation file in the JSONL format that will be consumed by Llama Index & Prompt Flow's evaluation flows.

In [ ]:
counter = 0
for document in documents:
    counter = counter + 1
    rag_data_generator = RagDatasetGenerator.from_documents(
        documents=[document], 
        show_progress=True,
        llm=get_aoai()['llm'],
        num_questions_per_chunk=2
    )    
    dataset = rag_data_generator.generate_dataset_from_nodes()
    input_file = f"{llama_path}/{document.doc_id}.jsonl"
    output_file = f"{pf_path}/{document.doc_id}.jsonl"
    dataset.save_json(input_file)
    convert_llama_to_pf_jsonl(input_file, output_file)
    print(f'[{counter}]: JSONL evaluation for LlamaIndex created at {input_file}')
    print(f'[{counter}]: JSONL evaluation for PromptFlow created at {output_file}')   
    # display(JSON(f"{dataset.json()}"))

### 2d. Merge all jsonl files into 1 single file

This merged file will be used as test data for our evaluation flows.

In [ ]:
%run -i ../utilities/jsonlConverter.py
merge_jsonl_files(pf_path, pf_merged_path)